# 1. Import Dependencies

In [3]:
# import GYM
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# import helpers
import numpy as np
import random
import os

# import stable-baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [5]:
action_space = Discrete(3)

In [10]:
action_space.sample()

2

In [11]:
Box(0,1,shape = (3,3))

Box([[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], (3, 3), float32)

In [12]:
Box(0,1,shape = (3,3)).sample()

array([[0.62344307, 0.55425674, 0.5740226 ],
       [0.5307312 , 0.5378462 , 0.54567355],
       [0.76929855, 0.39540967, 0.6026383 ]], dtype=float32)

In [13]:
Box(0,1,shape = (3,)).sample()

array([0.9909264 , 0.6253338 , 0.35185286], dtype=float32)

In [16]:
Tuple((Discrete(3),Box(0,1,shape = (3,))))# currently(2021) stable-baselines doesn't support Tuple

Tuple(Discrete(3), Box([0. 0. 0.], [1. 1. 1.], (3,), float32))

In [17]:
Tuple((Discrete(3),Box(0,1,shape = (3,)))).sample()

(2, array([0.6653522 , 0.95326453, 0.90177464], dtype=float32))

In [18]:
Dict({'height':Discrete(2),"speed":Box(0,100,shape=(1,))})

Dict(height:Discrete(2), speed:Box([0.], [100.], (1,), float32))

In [19]:
Dict({'height':Discrete(2),"speed":Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([86.53502], dtype=float32))])

In [20]:
MultiBinary(4)

MultiBinary(4)

In [21]:
MultiBinary(4).sample()

array([0, 1, 0, 0], dtype=int8)

In [22]:
MultiDiscrete([5,2,2])

MultiDiscrete([5 2 2])

In [34]:
MultiDiscrete([5,2,2]).sample()#0-4,0-1,0-1

array([2, 0, 0], dtype=int64)

# 3.  Building an Environment
- Build an agent to give use the best shower possible
- Randomly tempreture
- 37 to 39 degrees

In [37]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = np.array([0]),high = np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
    def step(self,action):
        # Apply temp adj
        self.state += action - 1
        
        # Discrete shower time
        self.shower_length -= 1
        
        # Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state,reward,done,info
        
    def render(self):
        # Implement viz
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [38]:
env = ShowerEnv()

D:\anaconda3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [39]:
env.observation_space

Box([0.], [100.], (1,), float32)

In [40]:
env.action_space

Discrete(3)

In [41]:
env.reset()

array([38.])

# 4. Test Environment

In [50]:
episodes=5
for episode in range(1,episodes+1):
    obs=env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs,reward,done,info=env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:-26
Episode:2 Score:-38
Episode:3 Score:-60
Episode:4 Score:-48
Episode:5 Score:34


# 5. Train Model

In [51]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [52]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -30.2    |
| time/              |          |
|    fps             | 338      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------


In [53]:
model.learn(total_timesteps=50000)

Logging to Training\Logs\PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -28.9    |
| time/              |          |
|    fps             | 867      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -29.6       |
| time/                   |             |
|    fps                  | 597         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007904125 |
|    clip_fraction        | 0.0962      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000346    |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -27.4       |
| time/                   |             |
|    fps                  | 511         |
|    iterations           | 11          |
|    time_elapsed         | 44          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.009036284 |
|    clip_fraction        | 0.106       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.05       |
|    explained_variance   | 5.68e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 38.7        |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00719    |
|    value_loss           | 74.8        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 13.9        |
| time/                   |             |
|    fps                  | 514         |
|    iterations           | 21          |
|    time_elapsed         | 83          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.010366503 |
|    clip_fraction        | 0.17        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.05       |
|    explained_variance   | -0.000728   |
|    learning_rate        | 0.0003      |
|    loss                 | 21.4        |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.0151     |
|    value_loss           | 36.4        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

# 6. Save Model

In [54]:
shower_path = os.path.join('Training','Saved Models','Shower_Model_PPO')
model.save(shower_path)

In [56]:
evaluate_policy(model,env,n_eval_episodes=10,render=False)

(59.4, 0.9165151389911679)